In [1]:
# Dependencies
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from json import loads

In [2]:
# Setup Kafka and Spark Streaming
KAFKA_TOPIC = "social_media_topic"
BOOTSTRAP_SERVER = "localhost:9092"

ssc = StreamingContext(sc, 1) #stream each one second
ssc.checkpoint("./checkpoint")
lines = KafkaUtils.createDirectStream(ssc, [KAFKA_TOPIC],
                                      {"metadata.broker.list": BOOTSTRAP_SERVER})

In [3]:
def calculate_data(lines, window=60, sliding=60):
    def map_elements(line):
        el = loads(line[1])
        data = {}
        socmed_type = el["crawler_target"]["specific_resource_type"]
        timestamp = None
        
        if socmed_type == "twitter":
            timestamp = el["created_at"]
            data["user_count"] = 1
            data["stream_count"] = 1
        elif socmed_type == "youtube":
            timestamp = el["snippet"]["publishedAt"]
            data["user_count"] = 1
            data["stream_count"] = 1
        elif socmed_type == "instagram":
            timestamp = el["created_time"]
            data["user_count"] = 1
            data["stream_count"] = 1
        elif socmed_type == "facebook":
            timestamp = el["created_time"]
            data["user_count"] = 1
            data["stream_count"] = 1

        return ((socmed_type, timestamp), data)
    
    def map_result(result):
        return lines.map(map_elements)
    
    # Streaming Main
    process_result = lines.map(map_result)
    
    return process_result
 
social_media = calculate_data(lines)
social_media.pprint()

ssc.start()
ssc.awaitTermination()

Traceback (most recent call last):
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/serializers.py", line 597, in dumps
    return cloudpickle.dumps(obj, 2)
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/home/bigdata/anaconda3/lib/python3.7/pickle.py", line 437, in dump
    self.save(obj)
  File "/home/bigdata/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/bigdata/anaconda3/lib/python3.7/pickle.py", line 774, in save_tuple
    save(element)
  File "/home/bigdata/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 400, in save_function
    self.s

Py4JJavaError: An error occurred while calling o33.start.
: java.io.IOException: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/serializers.py", line 597, in dumps
    return cloudpickle.dumps(obj, 2)
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/home/bigdata/anaconda3/lib/python3.7/pickle.py", line 437, in dump
    self.save(obj)
  File "/home/bigdata/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/bigdata/anaconda3/lib/python3.7/pickle.py", line 774, in save_tuple
    save(element)
  File "/home/bigdata/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call un